In [4]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from typing import TypedDict, Literal, Annotated
from dotenv import load_dotenv
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_groq import ChatGroq
from langgraph.checkpoint.memory import InMemorySaver
import operator

In [5]:
load_dotenv()

True

In [6]:
llm = ChatGroq(model="Gemma2-9b-It",temperature=0)

In [7]:
class JokeState(TypedDict):
    topic: str
    joke: str
    explanation: str

In [8]:
def generate_joke(state: JokeState):
    prompt = f"Generate a joke on the topic: {state['topic']}"
    response = llm.invoke(prompt).content
    
    return {
        'joke': response
    }

In [9]:
def generate_explanation(state: JokeState):
    prompt = f"Write an explanation for the joke - {state['joke']}"
    response = llm.invoke(prompt).content
    
    return {
        'explanation': response
    }

In [10]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)

In [11]:
config1 = {
    'configurable': {"thread_id": "1"}
}

workflow.invoke(
    {'topic': 'pizza'},
    config=config1
)

{'topic': 'pizza',
 'joke': 'Why did the pizza slice go to the doctor? \n\nBecause it was feeling a little cheesy! 🍕🩺  \n',
 'explanation': 'This joke plays on the double meaning of the word "cheesy." \n\n* **Literal meaning:** Pizza is made with cheese, so a pizza slice literally *is* cheesy.\n\n* **Figurative meaning:**  "Cheesy" can also mean something that is corny, unoriginal, or embarrassing.  \n\nThe joke sets you up to think about the pizza slice being literally made of cheese. Then, it twists the meaning to suggest the pizza slice is feeling embarrassed or awkward, just like something "cheesy" might make you feel.  \n\n\nLet me know if you\'d like to hear another joke! 😄 \n'}

In [12]:
list(workflow.get_state(config1))

[{'topic': 'pizza',
  'joke': 'Why did the pizza slice go to the doctor? \n\nBecause it was feeling a little cheesy! 🍕🩺  \n',
  'explanation': 'This joke plays on the double meaning of the word "cheesy." \n\n* **Literal meaning:** Pizza is made with cheese, so a pizza slice literally *is* cheesy.\n\n* **Figurative meaning:**  "Cheesy" can also mean something that is corny, unoriginal, or embarrassing.  \n\nThe joke sets you up to think about the pizza slice being literally made of cheese. Then, it twists the meaning to suggest the pizza slice is feeling embarrassed or awkward, just like something "cheesy" might make you feel.  \n\n\nLet me know if you\'d like to hear another joke! 😄 \n'},
 (),
 {'configurable': {'thread_id': '1',
   'checkpoint_ns': '',
   'checkpoint_id': '1f07114a-a921-6d87-8002-669ba947ca07'}},
 {'source': 'loop', 'step': 2, 'parents': {}},
 '2025-08-04T09:23:22.347251+00:00',
 {'configurable': {'thread_id': '1',
   'checkpoint_ns': '',
   'checkpoint_id': '1f07114a

In [13]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza slice go to the doctor? \n\nBecause it was feeling a little cheesy! 🍕🩺  \n', 'explanation': 'This joke plays on the double meaning of the word "cheesy." \n\n* **Literal meaning:** Pizza is made with cheese, so a pizza slice literally *is* cheesy.\n\n* **Figurative meaning:**  "Cheesy" can also mean something that is corny, unoriginal, or embarrassing.  \n\nThe joke sets you up to think about the pizza slice being literally made of cheese. Then, it twists the meaning to suggest the pizza slice is feeling embarrassed or awkward, just like something "cheesy" might make you feel.  \n\n\nLet me know if you\'d like to hear another joke! 😄 \n'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f07114a-a921-6d87-8002-669ba947ca07'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-08-04T09:23:22.347251+00:00', parent_config={'configurable': {'thread_id': '1', '

In [22]:
config2 = {
    'configurable': {"thread_id": "2"}
}

workflow.invoke(
    {'topic': 'Broast'},
    config=config2
)

{'topic': 'Broast',
 'joke': "Why did the broast chicken cross the road? \n\nTo prove he wasn't chicken! 🐔🍗  \n\n\nLet me know if you'd like to hear another one! 😄 \n",
 'explanation': 'This joke plays on the double meaning of the word "chicken." \n\n* **Literal meaning:** A chicken is a type of bird.\n* **Figurative meaning:**  "Chicken" can also mean someone who is cowardly or afraid to do something.\n\nThe joke sets up the expectation of a simple animal crossing the road, but then subverts it by using the word "chicken" in its figurative sense.  The punchline implies that the roast chicken was brave enough to cross the road, defying the stereotype of chickens being afraid.  \n\n\nLet me know if you\'d like to hear another joke! 😄  \n\n'}

In [23]:
list(workflow.get_state(config2))

[{'topic': 'Broast',
  'joke': "Why did the broast chicken cross the road? \n\nTo prove he wasn't chicken! 🐔🍗  \n\n\nLet me know if you'd like to hear another one! 😄 \n",
  'explanation': 'This joke plays on the double meaning of the word "chicken." \n\n* **Literal meaning:** A chicken is a type of bird.\n* **Figurative meaning:**  "Chicken" can also mean someone who is cowardly or afraid to do something.\n\nThe joke sets up the expectation of a simple animal crossing the road, but then subverts it by using the word "chicken" in its figurative sense.  The punchline implies that the roast chicken was brave enough to cross the road, defying the stereotype of chickens being afraid.  \n\n\nLet me know if you\'d like to hear another joke! 😄  \n\n'},
 (),
 {'configurable': {'thread_id': '2',
   'checkpoint_ns': '',
   'checkpoint_id': '1f071198-677c-68fe-8002-11c31e353b0b'}},
 {'source': 'loop', 'step': 2, 'parents': {}},
 '2025-08-04T09:58:09.260364+00:00',
 {'configurable': {'thread_id': '

In [24]:
list(workflow.get_state_history(config2))

[StateSnapshot(values={'topic': 'Broast', 'joke': "Why did the broast chicken cross the road? \n\nTo prove he wasn't chicken! 🐔🍗  \n\n\nLet me know if you'd like to hear another one! 😄 \n", 'explanation': 'This joke plays on the double meaning of the word "chicken." \n\n* **Literal meaning:** A chicken is a type of bird.\n* **Figurative meaning:**  "Chicken" can also mean someone who is cowardly or afraid to do something.\n\nThe joke sets up the expectation of a simple animal crossing the road, but then subverts it by using the word "chicken" in its figurative sense.  The punchline implies that the roast chicken was brave enough to cross the road, defying the stereotype of chickens being afraid.  \n\n\nLet me know if you\'d like to hear another joke! 😄  \n\n'}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f071198-677c-68fe-8002-11c31e353b0b'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-08-04T09:58:09.260364+00